In [1]:
import pandas as pd
import json
import re
import os

In [2]:
data = pd.read_csv("../../data/FAA_data/Maintenance_Text_data_nona.csv")
text_col = "c119"
text = list(data[text_col])

In [5]:
len(text)

2748

In [6]:
def tokenize(sentence):
    # Define the special characters to be treated as separate tokens
    special_characters = [".", ",", '"', "(", ")", "[", "]", "{", "}"] # minus apostrophe, which has to be handled separately

    # Split the sentence using whitespace characters
    words = re.split(r'\s+', sentence)

    # Initialize an empty list to store the final tokens
    tokens = []

    # Iterate through each word and further split based on special characters
    for word in words:
        word = word.lower()

        # Split the word based on special characters
        split_word = re.split(f'([{re.escape("".join(special_characters))}])', word)

        # Check for apostrophe:
        for wd in split_word:
            # Check if the word contains an apostrophe followed by an 's'
            if "'s" in wd:
                # Split the word into two tokens: the word without "'s" and "'s"
                tokens.extend(re.split(r"('s)", wd))
            elif "n't" in wd:
                tokens.extend(re.split(r"(n't)", wd))
            elif "'ll" in wd:
                tokens.extend(re.split(r"('ll)", wd))
            elif "'re" in wd:
                tokens.extend(re.split(r"('re)", wd))
            elif "'m" in wd:
                tokens.extend(re.split(r"('m)", wd))
            elif "'d" in wd:
                tokens.extend(re.split(r"('d)", wd))
            elif "'ve" in wd:
                tokens.extend(re.split(r"('ve)", wd))
            elif "'" in wd:
                tokens.extend(re.split(r"'", wd))
            else:
                tokens.append(wd)

    # Filter out empty strings from the tokens list
    tokens = list(filter(None, tokens))

    return tokens

In [26]:
split_text = []
for entry in text:
    if '.' in entry:
        sentences = []
        for sentence in entry.split('.'):
            if not sentence.isspace() and len(sentence) > 0:
                tokens = tokenize(sentence + '.')
                sentences.append(tokens)
        split_text.append(sentences)
    else:
        tokens = tokenize(entry)
        split_text.append([tokens])

In [27]:
len(split_text)

2748

In [39]:
formatted = [{'sentences':[sentence for sentence in doc], 'ner':[[] for sentence in doc], 'relations':[[] for sentence in doc], 'doc_key':f'faa_{idoc}'} for idoc, doc in enumerate(split_text)]

In [40]:
with open("./PL-Marker/scierc/faa_eval_faa.json", "w") as json_file:
    for i in range(len(formatted)):
        formatted_str = json.dumps(formatted[i]) + '\n'
        json_file.write(formatted_str)